In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn 
from sklearn.datasets import make_regression
from torch.utils.data import Dataset, DataLoader
import torch
from typing import Tuple
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


In [2]:
# Make linear regression problem
X_raw, y_raw = make_regression(
    n_features=10,
    n_informative=5,
    n_samples=10000,

)
# Standard scale X and y
X_scaler = StandardScaler()
X_np = X_scaler.fit_transform(X_raw)

y_scaler = StandardScaler()
y_np = y_scaler.fit_transform(y_raw.reshape(-1,1)).flatten()

# Fit a linear regression to check coef
linear_regression = LinearRegression()
linear_regression.fit(X_np, y_np)
linear_regression.coef_

array([-3.06058589e-16, -1.66533454e-16,  7.40578140e-17,  5.94118465e-01,
        3.37884065e-01, -3.68313332e-16,  3.60265228e-01,  1.25244403e-01,
        6.29419989e-01, -1.35217446e-16])

In [3]:
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32)

In [4]:
X

tensor([[-0.1358, -0.9549, -0.6626,  ...,  0.0371, -0.7060,  1.0937],
        [ 0.9668, -1.5644, -0.3024,  ..., -0.0902,  0.4332,  0.8637],
        [ 1.6658, -0.0986,  0.8327,  ...,  0.4785,  0.9229, -1.2112],
        ...,
        [ 0.8226, -0.1252, -1.5468,  ...,  0.2786, -0.0556,  0.2661],
        [ 0.5509,  0.2108, -1.6172,  ..., -0.2233, -1.0985,  1.3313],
        [ 0.4317,  0.3193, -0.0791,  ...,  0.9349, -2.8498, -0.4877]])

In [5]:
y

tensor([ 0.4136,  0.9394,  1.1275,  ..., -0.0064, -0.8932, -1.8407])

In [6]:
class Model(nn.Module):
    def __init__(
            self,
            n_features: int,
    ) -> None:
        super().__init__()
        self.fc1 = nn.Linear(
            in_features=n_features,
            out_features=1,
        )

    def forward(
        self,  
        X: torch.Tensor,  
    ) -> torch.Tensor:
        y = self.fc1(X)
        return y.flatten()
        

In [7]:
class CustomLoss(nn.Module):
    def __init__(
            self,
            threshold: float = 0.3,
            weight_max_error: float = 2,
            weight_percentage_above_threshold:float = 3,
            weight_wrong_sign: float = 1,
            sigmoid_steepness: float = 100,
    ) -> None:
        super().__init__()
        self.steepness = sigmoid_steepness
        self.threshold = threshold
        # Normalize weights and assign them
        sum_weights = (
            weight_max_error
            + weight_percentage_above_threshold
            + weight_wrong_sign
        )
        self.weight_max_error = (
            weight_max_error / sum_weights
        )
        self.weight_percentage_above_threshold = (
            weight_percentage_above_threshold / sum_weights
        )
        self.weight_wrong_sign = (
            weight_wrong_sign / sum_weights
        )

    def forward(
            self, 
            inputs: torch.Tensor, 
            targets: torch.Tensor,
        ) -> torch.Tensor:

        residuals = targets - inputs
        # Maximum abs error
        max_error = residuals.abs().max()

        # Percentage of time above threshold value
        percentage_of_time_above_x = (
            1/(1+torch.e**(-self.steepness*(residuals.abs()-self.threshold)))
        ).mean()

        # Percentage of time wrong sign
        loss_percentage_of_time_wrong_sign = (
            1/(1+torch.e**(-self.steepness*(inputs*targets)))
        )
        
        # Total loss
        total_loss = (
            self.weight_max_error * max_error
            + self.weight_percentage_above_threshold * percentage_of_time_above_x
            + self.weight_wrong_sign * loss_percentage_of_time_wrong_sign
        )
        return percentage_of_time_above_x

In [8]:
class CustomDataset(Dataset):
    def __init__(
            self,
            X: torch.Tensor,
            y: torch.Tensor,
        ) -> None:
        self.X = X
        self.y = y

    def __len__(
            self
    ) -> int:
        return self.X.shape[0]
    
    def __getitem__(
            self,
            idx: int,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        X_item = self.X[idx,:]
        y_item = self.y[idx]
        return X_item, y_item

In [9]:
epochs = 100
lr = 1e-4
batch_size = 10

dataloader = DataLoader(
    CustomDataset(X,y),
    batch_size=batch_size,
    shuffle=True,
)
model = Model(10)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = lr,
)
criterion = CustomLoss(
    weight_wrong_sign=10,
    weight_max_error=1,
    weight_percentage_above_threshold=1,
)
for epoch in range(epochs):
    epoch_loss = 0
    for i,(X_batch, y_batch) in enumerate(dataloader):
        prediction = model(X_batch)
        optimizer.zero_grad()
        loss = criterion(prediction, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        average_loss = epoch_loss/(i+1e-10)
    print(f"Average epoch loss: {average_loss}")
    print(f"Epoch {epoch} done.")

Average epoch loss: 0.7387279734178802
Epoch 0 done.
Average epoch loss: 0.7305702490431156
Epoch 1 done.
Average epoch loss: 0.7222951591252796
Epoch 2 done.
Average epoch loss: 0.7133668655150408
Epoch 3 done.
Average epoch loss: 0.7026615778843097
Epoch 4 done.
Average epoch loss: 0.6909708274868898
Epoch 5 done.
Average epoch loss: 0.678041777796267
Epoch 6 done.
Average epoch loss: 0.6625545976218571
Epoch 7 done.
Average epoch loss: 0.6454553684224665
Epoch 8 done.
Average epoch loss: 0.62479743239155
Epoch 9 done.
Average epoch loss: 0.5986939884117533
Epoch 10 done.
Average epoch loss: 0.5639153707014938
Epoch 11 done.
Average epoch loss: 0.5151876770814754
Epoch 12 done.
Average epoch loss: 0.44980618797025584
Epoch 13 done.
Average epoch loss: 0.334912649820095
Epoch 14 done.
Average epoch loss: 0.1626605201395298
Epoch 15 done.
Average epoch loss: 0.025229118747941086
Epoch 16 done.
Average epoch loss: 0.002163916513218864
Epoch 17 done.
Average epoch loss: 0.000533696943042

In [10]:
model.fc1.weight

Parameter containing:
tensor([[-1.4661e-04, -4.2382e-05, -5.5316e-04,  5.9068e-01,  3.3665e-01,
         -5.8377e-04,  3.5941e-01,  1.2512e-01,  6.2731e-01, -2.5383e-04]],
       requires_grad=True)

In [11]:
linear_regression.coef_

array([-3.06058589e-16, -1.66533454e-16,  7.40578140e-17,  5.94118465e-01,
        3.37884065e-01, -3.68313332e-16,  3.60265228e-01,  1.25244403e-01,
        6.29419989e-01, -1.35217446e-16])

# Tryout some loss functions

In [12]:
pred = torch.rand(5, requires_grad=True) - 0.5
real = torch.rand(5, requires_grad=True) - 0.5
residuals = real - pred

In [13]:
residuals

tensor([ 0.4042,  0.0645, -0.6739, -0.1173, -0.7771], grad_fn=<SubBackward0>)

In [14]:
real

tensor([ 0.2399,  0.2546, -0.4177,  0.3813, -0.4837], grad_fn=<SubBackward0>)

In [15]:
pred

tensor([-0.1643,  0.1901,  0.2562,  0.4986,  0.2934], grad_fn=<SubBackward0>)

In [16]:
steepness = 100 
threshold = 0.3

loss1 = 1/(1+torch.e**(-steepness*(residuals-threshold)))
loss1

tensor([9.9997e-01, 5.9357e-11, 0.0000e+00, 7.5487e-19, 0.0000e+00],
       grad_fn=<MulBackward0>)

In [17]:
signs = pred*real / (pred*real).abs()
loss2 = 1/(1+torch.e**(-steepness*(signs)))
loss2

tensor([0., 1., 0., 1., 0.], grad_fn=<MulBackward0>)

In [18]:
pred*real 

tensor([-0.0394,  0.0484, -0.1070,  0.1901, -0.1419], grad_fn=<MulBackward0>)